In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import plotly.graph_objects as go

In [3]:
demand = pd.read_excel('../dataset/a6_booster_delivery_planning.xlsx')
#events = pd.read_excel('../dataset/ariane_tracks_booster_container_1.0.xlsx')
events = pd.read_excel('../dataset/ariane_tracks_booster_container_2.0.xlsx')
events.drop(columns=['FM'],inplace=True)
dataset = pd.read_csv('../dataset/true_datas.csv')
clean_data = pd.read_csv('../dataset/clean_data_paul.csv')

states = ['A','B','C','D','E']
containers = ['SN'+str(i) for i in range(1,9)]
events['Entry'] = events['Entry'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
events['Exit'] = events['Exit'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))


def days_spent_for_container(container):
    return({state:[(row[3] - row[2]).days for row in events.loc[(events['Container'] == container) & (events['Zone'] == state)].values] for state in states})

## I.   Data viz

In [101]:
print("demand shape :", demand.shape)
demand.head()

demand shape : (39, 2)


,Date,Config
0,2021-01-12,2
1,2021-08-24,4
2,2021-10-19,2
3,2022-01-28,2
4,2022-06-21,4


In [102]:
print("events shape :", events.shape)
events.head(50)

events shape : (353, 4)


,Container,Zone,Entry,Exit
0,SN6,A,2020-12-09,2020-12-24
1,SN6,B,2020-12-24,2020-12-29
2,SN6,C,2020-12-29,2020-12-30
3,SN6,D,2021-01-10,2021-01-12
4,SN6,E,2021-01-12,2021-01-27
5,SN6,D,2021-01-27,2021-01-29
6,SN6,C,2021-02-12,2021-02-13
7,SN6,B,2021-02-13,2021-02-18
8,SN5,A,2020-12-09,2020-12-24
9,SN5,B,2020-12-24,2020-12-29


## II.    correction algorithm

In [4]:
def fetch_same_date(data, entry):
    df = data.loc[data['Entry']==entry]
    if df.empty:
        return("ok", "ok", False)
    return(df['Entry'].iloc[0], df['Exit'].iloc[0], True)


def complete_steps(data):
    sorted_events = data.sort_values(['Exit'])
    cycle = ['A','B','C','D','E','D','C','B']
    values = []
    for container in containers:
        df = sorted_events.loc[(sorted_events['Container'] == container)]
        new_values = []
        step = 0
        for i,line in enumerate(df.values):

            while line[1] != cycle[step%8]:
                container_ = container
                state_ = cycle[step%8]
                entry_, exit_, result = fetch_same_date(data, df.values[i-1][3])
                print("corrupted_data")
                print(line)
                if not result:
                    entry_ = "ok"
                    exit_ = "ok"
                
                new_values.append([container_, state_, entry_, exit_])
                step+=1

            step+=1
            new_values.append(line)
        values.extend(new_values)
    
    return(pd.DataFrame(values, columns=['Container', 'Zone', 'Entry', 'Exit']))



def sanity_checking(data):
    data_ = data.explode('Container').sort_values(['Exit'])
    states = ['A','B','C','D','E','D','C','B']
    c=0
    for container in containers:
        df = data_.loc[data_['Container']==container]
        pointer=0
        for line in df.values:
            if line[1]!=states[pointer]:
                print(line)
                c+=1

            pointer+=1
            if pointer==8:
                pointer=0
    return(c)
data = complete_steps(events)
print("errors for containers :", sanity_checking(data))

corrupted_data
['SN5' 'A' Timestamp('2021-12-26 00:00:00')
 Timestamp('2022-01-10 00:00:00')]
errors for containers : 0


In [ ]:
#data.to_csv('../dataset/containers.csv',index=False)

## III. ML Model

In [5]:
dataset.head()

,Days,Inverted,PerfMean,PerfStd,EntryLoad,ExitLoad,EntryDemand,ExitDemand,Distance,ContainerQuantity,...,Entry_snowfall_sum,Entry_wind_speed_10m_max,Entry_wind_gusts_10m_max,Entry_wind_direction_10m_dominant,Exit_precipitation_sum,Exit_rain_sum,Exit_snowfall_sum,Exit_wind_speed_10m_max,Exit_wind_gusts_10m_max,Exit_wind_direction_10m_dominant
0,15.0,0.0,24.0,7.348469,0.0,0.0,0.0,0.0,0.00,2.0,...,0.0,26.980793,63.00,247.30750,0.0,0.0,0.0,21.897945,42.12,343.37125
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,9.70,2.0,...,0.0,21.897945,42.12,343.37125,0.0,0.0,0.0,21.897945,42.12,343.37125
2,5.0,0.0,5.0,0.000000,0.0,0.0,0.0,0.0,0.00,2.0,...,0.0,21.897945,42.12,343.37125,3.7,3.7,0.0,19.174856,36.36,242.80690
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,9.42,2.0,...,0.0,19.174856,36.36,242.80690,3.7,3.7,0.0,19.174856,36.36,242.80690
4,1.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.00,2.0,...,0.0,19.174856,36.36,242.80690,17.6,17.6,0.0,21.096123,39.96,251.11560


In [6]:
y = dataset.Days
X = dataset.drop(columns=['Days','Entry','Exit'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [8]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("mean squared error : ", mse)

mean squared error :  497.75577892740773


In [10]:
feature_rename = {
'Inverted':'Direction',
'PerfMean':'Average stage duration', 
'PerfStd':'Deviation of stage duration', 
'EntryLoad':'Entry load', 
'ExitLoad':'Exit load',
'EntryDemand':'Entry demand',
'ExitDemand':'Exit demand',
'Distance':'Distance',
'ContainerQuantity':'Container quantity',
'Stage':'Stage',
'Entry_Working_Day':'Entry working day',
'Exit_Working_Day':'Exit working day',
'Entry_precipitation_sum':'Entry precipitation',
'Entry_rain_sum':'Entry rain',
'Entry_snowfall_sum':'Entry snowfall',
'Entry_wind_speed_10m_max':'Entry wind speedx',
'Entry_wind_gusts_10m_max':'Entry wind gusts',
'Entry_wind_direction_10m_dominant':'Entry wind direction',
'Exit_precipitation_sum':'Exit precipitation',
'Exit_rain_sum':'Exit rain',
'Exit_snowfall_sum':'Exit snowfall',
'Exit_wind_speed_10m_max':'Exit wind speed',
'Exit_wind_gusts_10m_max':'Exit wind gusts',
'Exit_wind_direction_10m_dominant':'Exit wind direction',
}


importances = model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X.columns.map(feature_rename),
    'Importance': importances
})


threshold = 0.001  # Set your threshold here
feature_importance_df = feature_importance_df[feature_importance_df['Importance'] >= threshold]

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Create bar chart
fig = go.Figure(go.Bar(
    x=feature_importance_df['Importance'],
    y=feature_importance_df['Feature'],
    orientation='h',
    marker=dict(color='#519EE5')
))

# Update layout
fig.update_layout(title='Feature Importance',
                  xaxis_title='Importance',
                  yaxis_title='Feature',
                  yaxis=dict(automargin=True),
                  height=600,
                 )

# Show plot
fig.show()

In [194]:
data['Container'].to_csv('../folder_vizualisation/container_use.csv')

In [152]:
colors = ['#BFEEDB', '#8ED3AE', '#48AF8A', '#247B6C', '#CCE3F8', '#85C6EB', '#519EE5', '#283952']

# Data
containers = ['container '+str(i) for i in range(1,9)]
quantites_containers = list(data['Container'].value_counts())

# Create figure
fig = go.Figure(data=[go.Pie(labels=containers, values=quantites_containers)])

# Update layout
fig.update_traces(marker=dict(colors=colors))
fig.update_layout(title='Usage of containers')

# Show plot
fig.show()

In [173]:
stage_state = {
    0:'A',
    1:'AB',
    2:'B',
    3:'BC',
    4:'C',
    5:'CD',
    6:'D',
    7:'DE',
    8:'E',
}
df = pd.concat([clean_data['Days'], clean_data['Stage'].map(stage_state)], axis=1)
df.loc[df['Stage']=='E'].head(30)

,Days,Stage
8,15,E
24,15,E
41,15,E
57,15,E
74,15,E
92,15,E
109,15,E
125,15,E
141,15,E
156,0,E


In [187]:
etats = list(stage_state.values())
donnees_par_etat = []

# Parcourir chaque état et obtenir le nombre de fois qu'il apparaît dans la colonne 'Days'
for etat in etats:
    nombre_apparitions = df.loc[df['Stage'] == etat]['Days']
    donnees_par_etat.append(nombre_apparitions)

# Créer le diagramme à moustaches avec Plotly
fig = go.Figure()

for i in range(len(etats)):
    fig.add_trace(go.Box(y=donnees_par_etat[i], name=etats[i]))

fig.update_layout(title='Distribution du nombre d\'apparitions pour chaque état',
                  xaxis=dict(title='États'),
                  yaxis=dict(title='Nombre d\'apparitions'),
                  boxmode='group')

fig.show()

In [192]:
etats = list(stage_state.values())

figs = []

# Parcourir chaque état et créer un diagramme à moustaches pour chaque état
for etat in etats:
    donnees_etat = df.loc[df['Stage'] == etat]['Days']
    fig = go.Figure()
    fig.add_trace(go.Box(y=donnees_etat, name=etat))
    fig.update_layout(height=300, width=400,
                      title=f'Distribution du nombre d\'apparitions pour l\'état {etat}',
                      xaxis=dict(title='États'),
                      yaxis=dict(title='Nombre d\'apparitions'))
    figs.append(fig)

# Afficher chaque diagramme à moustaches séparément
for fig in figs:
    fig.show()